# Infinite Investment Systems Predictive Customer Churn Model
Using real data on client portfolio accounts from Infinite Investment Systems, this model predicts the customer churn

### Context
As a person pursuing the field of data analytics (not science), I don't have much experience building machine learning model, so I took this opportunity to further hone my skills. I learned a lot about tree-based algorithms and mitigating overfitting, which are common issues faced in this field. Overall, I thoroughly enjoyed it!

**Note to person reading**:
As of the time of this writing, it is currently 5:11 AM (5 hours past the deadline and I am about to submit) and I spent the past 9 hours straight coding this project. The reason I spent the extra 5 hours is because technically, I finished the model 30 min before the deadline, but I didn't do the actual predicting on test.csv, which I thought would take 5 minutes (not 5 hours). I worked until the coffee shop closed at 12 and I had to worked on it at a hospital lobby until 2am for me to charge my laptop. Then, on my way back home, on the TTC subway on a friday night, I was still working non-stop and kept working a couple hours more when I got home. I quite frankly thought of giving up at least 30 times, but something inside me kept pushing me to keep going. And because of my relentless perserverence, this was probably the most cruel, yet rewarding project I did so far. So, as much as I understand that I missed the deadline, it would be greatly appreciated if it can still be considered in the competition. Thanks!

### Resources I used
- https://towardsdatascience.com/3-techniques-to-avoid-overfitting-of-decision-trees-1e7d3d985a09
- https://towardsdatascience.com/churn-prediction-with-machine-learning-ca955d52bd8c

# 0. Import libraries and dataset

In [513]:
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report, f1_score, roc_auc_score

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

In [514]:
df = pd.read_csv('history.csv')
df

,id,type_code,is_registered,country_code,currency_code,is_active,class_id,debit_code,last_trade_date,contract_type,inception_date,net_of_fees,cashflows_custody_fee,fee_paid_separately,custody_fee_withdrawal,is_fee_exempt,branch,include_client_consolidation,use_client_address,credit_limit_type,retail_plan,is_spousal,is_arp_locked,arp_pension_origin,language_code,dividend_confirm_code,sss_location,options_trading_type,sss_type,sss_agent,is_midwest_clearing_account,rep_commission_rate,rep_commission_override,interest_dividend_conversion_type,guarantee_gtor_type,use_hand_delivery,use_mail,share_name_address_to_issuer,shareholder_instructions_received,rrsp_limit_reached,is_portfolio_account,has_no_min_commission,is_tms_eligible,is_agent_bbs_participant,is_parameters_account,is_spousal_transfer,spousal_age_flag,terminal_code,has_multiple_name,deceased_fair_market_value,target_grantor_grantee_flag,discretionary_trading_authorized,iso_funds_code,esir_number,shareholder_language,dup_trip_quad_code,special_tag,conjunction,title,function_code,tms_settlement_location,portfolio_cost_method,portfolio_name_address_option,portfolio_report_option,portfolio_summary_option,receive_general_mailings,loan_limit_override,interactive_portfolio_code,non_plan_book_value_flag,has_discrete_auth,last_update_date,is_non_objecting_beneficial_owner,is_objecting_to_disclose_info,mailing_consent,consent_to_pay_for_mail,consent_to_email_delivery,has_received_instruction,risk_tolerance,investment_objective,is_broker_account,is_inventory_account,is_gl_account,is_control_account,is_extract_eligible,last_maintenance_time,last_maintenance_user,is_pledged,special_fee_code,non_calendar_year_end,plan_effective_date,plan_end_date,is_resp,number_of_beneficiaries,rrif_original_date,use_original_date_for_payment_calc,is_family_resp,is_hrdc_resp,is_plan_grandfathered,resp_specimen_plan,inserted_at,updated_at,is_olob,retail_last_maintenance_time,retail_last_maintenance_user,visible_in_reports,label
0,893724,CASH SWEEP,f,NaN,CAD,t,3.0,0,NaN,NaN,2017-08-18,f,NaN,f,f,f,IAVM,t,f,0.0,NaN,NaN,NaN,NaN,E,0.0,NaN,0.0,NaN,NaN,False,0.0,0.0,0.0,0.0,f,f,t,f,f,f,f,f,f,f,f,f,NaN,f,0.0,NaN,f,CAD,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,t,NaN,0.0,V,f,NaN,f,f,0.0,f,f,f,NaN,NaN,f,f,f,f,t,2021-12-16 00:00:00,T80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-30 14:20:18.531115+00,2023-08-08 18:53:01.439561+00,f,NaN,NaN,f,Churn
1,1268094,RRIF,t,NaN,CAD,t,5.0,C,2022-07-04,18.0,2018-11-08,f,NaN,f,f,f,IAVM,t,t,0.0,RI,f,f,NaN,E,0.0,NaN,0.0,NaN,NaN,False,0.0,0.0,0.0,2.0,f,f,t,f,f,f,f,f,f,f,f,f,BATCH,f,0.0,NaN,f,CAD,NaN,NaN,A,NaN,NaN,MR,NaN,NaN,0.0,0.0,0.0,0.0,t,NaN,0.0,NaN,f,2022-09-08,t,f,0.0,f,f,t,H10M90,B05G85S10,f,f,f,f,t,2023-01-03 00:00:00,BATCH,f,NaN,0000,2018-11-08,NaN,f,0.0,2018-11-08,f,f,f,f,NaN,2023-05-30 14:20:18.531115+00,2023-08-10 21:47:25.370403+00,f,2023-01-03 00:00:00,BATCH,t,Churn
2,606613,SPOUSAL RRSP,t,NaN,CAD,t,5.0,A,2018-07-05,16.0,2009-04-08,f,NaN,f,f,f,IAVM,f,f,0.0,RS,t,f,NaN,F,0.0,NaN,0.0,NaN,NaN,False,0.0,0.0,0.0,2.0,f,f,t,f,f,f,f,f,f,f,f,f,G023,f,0.0,NaN,f,CAD,NaN,F,A,NaN,NaN,MADAME,NaN,NaN,0.0,0.0,0.0,0.0,t,NaN,0.0,NaN,f,2014-04-23,f,t,0.0,t,f,t,H50M50S00,G50S50,f,f,f,f,t,2021-03-15 00:00:00,GUERINO,f,NaN,00/0,2009-04-08,2018-07-17,f,0.0,2009-04-08,f,f,f,f,NaN,2023-05-30 14:20:18.531115+00,2023-08-10 21:47:25.370403+00,f,2018-07-17 00:00:00,GUERINO,f,Churn
3,741930,CASH,f,NaN,CAD,t,3.0,T,NaN,12.0,2021-01-04,f,NaN,f,f,f,IAVM,t,t,0.0,NaN,NaN,NaN,NaN,E,0.0,NaN,0.0,NaN,NaN,False,0.0,0.0,0.0,0.0,f,f,t,f,f,f,f,f,f,f,f,f,113C,f,0.0,NaN,f,CAD,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,t,NaN,0.0,V,f,2021-01-04,t,f,0.0,f,f,t,M99,G99,f,f,f,f,t,2021-01-04 00:00:00,T80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-30 14:20:18.531115+00,2023-08-08 18:53:01.439561+00,f,NaN,NaN,t,Churn
4,1137922,CASH,f,NaN,CAD,t,3.0,T,NaN,17.0,2017-07-21,f,NaN,f,f,f,IAVM,t,f,0.0,NaN,NaN,NaN,NaN,E,0.0,NaN,0.0,NaN,NaN,False,0.0,0.0,0.0,0.0,f,f,t,f,f,f,f,f,f,f,f,f,BATCH,f

# 1. Dropping unnecessary columns
First of all, this is a massive dataset with 106 columns. So, it is important for us to get rid of all columns that provide no predictive power or doesn't fit the context.

These are the steps that will be taken to remove these columns:
- Removing columns with no relevance to the business context
- Removing constant columns
- Remove near-constant columns that have little to no predictive power
- Remove any remaining columns with little to no predictive power (just to make sure)

## 1.1 Removing columns with no relevance to the business context
These are columns that do provides little to no benefit to predicting churn and can potentially harm the results. For instance, we know that id doesn't help predict churn rates.

I also removed any form of dates. Although there is a possibility that the date range of a consumer account's behaviour may have some correlation with churn rate, the return I get from processing it is marginal and simply not worth the processing.

In [515]:
# Remove columns with no relevance to the business context
df.drop(columns=['id', 'last_trade_date', 'inception_date', 'last_update_date', 'last_maintenance_time',
                 'plan_effective_date', 'plan_end_date', 'rrif_original_date', 'inserted_at', 'updated_at',
                 'retail_last_maintenance_time'], inplace=True)

## 1.2 Removing constant columns
**Constant columns** are columns where all values are the same.

There are 2 types of constant columns I will remove:
- Completely empty (only contains NaN)
- Non-empty but consists of only one value

In [516]:
# Check for number of NaNs per column
# We see that cashflows_custody_fee has 673339 NaNs, which is the total amount of rows in the datasetb
for col in df.columns:
    print(col, df[col].isna().sum())

type_code 237
is_registered 0
country_code 669050
currency_code 0
is_active 0
class_id 3198
debit_code 3198
contract_type 95823
net_of_fees 0
cashflows_custody_fee 673339
fee_paid_separately 0
custody_fee_withdrawal 0
is_fee_exempt 0
branch 3198
include_client_consolidation 0
use_client_address 0
credit_limit_type 3198
retail_plan 316985
is_spousal 313694
is_arp_locked 313694
arp_pension_origin 632725
language_code 3198
dividend_confirm_code 3198
sss_location 668258
options_trading_type 3198
sss_type 668258
sss_agent 668273
is_midwest_clearing_account 3198
rep_commission_rate 3198
rep_commission_override 3198
interest_dividend_conversion_type 3198
guarantee_gtor_type 3198
use_hand_delivery 0
use_mail 0
share_name_address_to_issuer 0
shareholder_instructions_received 0
rrsp_limit_reached 0
is_portfolio_account 0
has_no_min_commission 0
is_tms_eligible 0
is_agent_bbs_participant 0
is_parameters_account 0
is_spousal_transfer 0
spousal_age_flag 0
terminal_code 77413
has_multiple_name 0
dec

In [517]:
# Check for value_count per column and look for columns with 673339 values
for col in df.columns:
    print(col, df[col].value_counts())

type_code type_code
CASH              190811
REG RRSP          114218
TFSA              108073
CASH SWEEP         80307
RRIF               38902
MRGN               32115
LIRA/LRSP/RLSP     28290
RESP               22726
SPOUSAL RRSP       21885
OFFBOOK            11982
SPOUSAL RRIF        8978
COD                 7748
MISSING             3198
SHORT               1584
RDSP                1137
HYPOTHECATED         445
LIRA                 413
CASHA                249
HEDGE                 30
LIF/RLIF/LRIF         11
Name: count, dtype: int64
is_registered is_registered
t    344633
f    328706
Name: count, dtype: int64
country_code country_code
CAN    4289
Name: count, dtype: int64
currency_code currency_code
CAD    634512
USD     38827
Name: count, dtype: int64
is_active is_active
t    629323
f     44016
Name: count, dtype: int64
class_id class_id
5.0     356334
3.0     278969
6.0      16825
10.0      3833
7.0       3431
         ...  
43.0         1
13.0         1
62.0         1
25.0   

In [518]:
# DROP CONSTANT COLUMNS
df.drop(columns=['cashflows_custody_fee'], inplace=True) # completely empty; only constains NaNs
df.drop(columns=['net_of_fees', 'custody_fee_withdrawal', 'is_fee_exempt', 'rrsp_limit_reached', 'rrsp_limit_reached',
                 'is_portfolio_account', 'has_no_min_commission', 'is_parameters_account', 'spousal_age_flag',
                 'is_broker_account', 'is_gl_account', 'is_control_account', 'fee_paid_separately', 'special_fee_code',
                'portfolio_name_address_option', 'portfolio_summary_option', 'debit_code', 'sss_location'], inplace=True)
df.shape

(673339, 77)

## 1.3 Remove near-constant columns that have little to no predictive power

### 1.3.1 Gather near-constant columns
First, we will gather all near-constant columns. So let's say columns with 640,000+ NaNs or 600,000+ single values.

In [519]:
# Gather all near-constant columns
df_near_const = pd.DataFrame() # df that contains all near-constant columns
for col in df.columns:
    if df[col].isna().sum() > 640000 or df[col].value_counts().iloc[0] > 600000:
        df_near_const[col] = df[col]
df_near_const

,country_code,currency_code,is_active,include_client_consolidation,credit_limit_type,dividend_confirm_code,options_trading_type,sss_type,sss_agent,is_midwest_clearing_account,rep_commission_rate,rep_commission_override,interest_dividend_conversion_type,use_hand_delivery,use_mail,share_name_address_to_issuer,shareholder_instructions_received,is_tms_eligible,is_agent_bbs_participant,is_spousal_transfer,has_multiple_name,deceased_fair_market_value,iso_funds_code,esir_number,dup_trip_quad_code,conjunction,function_code,tms_settlement_location,portfolio_cost_method,portfolio_report_option,receive_general_mailings,loan_limit_override,interactive_portfolio_code,mailing_consent,consent_to_pay_for_mail,consent_to_email_delivery,is_inventory_account,is_extract_eligible,resp_specimen_plan,is_olob
0,NaN,CAD,t,t,0.0,0.0,0.0,NaN,NaN,False,0.0,0.0,0.0,f,f,t,f,f,f,f,f,0.0,CAD,NaN,A,NaN,NaN,NaN,0.0,0.0,t,NaN,0.0,0.0,f,f,f,t,NaN,f
1,NaN,CAD,t,t,0.0,0.0,0.0,NaN,NaN,False,0.0,0.0,0.0,f,f,t,f,f,f,f,f,0.0,CAD,NaN,A,NaN,NaN,NaN,0.0,0.0,t,NaN,0.0,0.0,f,f,f,t,NaN,f
2,NaN,CAD,t,f,0.0,0.0,0.0,NaN,NaN,False,0.0,0.0,0.0,f,f,t,f,f,f,f,f,0.0,CAD,NaN,A,NaN,NaN,NaN,0.0,0.0,t,NaN,0.0,0.0,t,f,f,t,NaN,f
3,NaN,CAD,t,t,0.0,0.0,0.0,NaN,NaN,False,0.0,0.0,0.0,f,f,t,f,f,f,f,f,0.0,CAD,NaN,A,NaN,NaN,NaN,0.0,0.0,t,NaN,0.0,0.0,f,f,f,t,NaN,f
4,NaN,CAD,t,t,0.0,0.0,0.0,NaN,NaN,False,0.0,0.0,0.0,f,f,t,f,f,f,f,f,0.0,CAD,NaN,A,NaN,NaN,NaN,0.0,0.0,t,NaN,0.0,0.0,f,f,f,t,NaN,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673334,CAN,CAD,t,t,0.0,0.0,0.0,NaN,NaN,False,0.0,0.0,0.0,f,f,t,f,f,f,f,f,0.0,CAD,NaN,A,NaN,NaN,NaN,0.0,0.0,t,NaN,0.0,0.0,f,f,f,t,NaN,f
673335,NaN,CAD,f,t,0.0,0.0,0.0,NaN,NaN,False,0.0,0.0,0.0,f,f,t,f,f,f,f,f,0.0,CAD,NaN,A,NOT,NaN,NaN,0.0,0.0,t,NaN,0.0,0.0,f,f,f,t,NaN,f
673336,NaN,CAD,t,t,0.0,0.0,0.0,NaN,NaN,False,0.0,0.0,0.0,f,f,t,f,f,f,f,t,0.0,CAD,NaN,A,NaN,NaN,NaN,0.0,0.0,t,NaN,0.0,0.0,t,f,f,t,NaN,f
673337,NaN,CAD,t,t,0.0,0.0,0.0,NaN,NaN,False,0.0,0.0,0.0,f,f,t,f,f,f,f,f,0.0,CAD,NaN,A,NaN,NaN,NaN,0.0,0.0,t,NaN,0.0,0.0,f,f,f,t,NaN,f


## 1.3.2 Apply churn analysis to determine predictive power
This is done by converting the labels into 1 & 0 and then calculating the mean per unique value. **We want to remove columns with similar churn rates, because it doesn't provide predictive power.**

For example, let's check if the 'country_code' feature have any predictive power by calculating the churn rate per unique value. We see that for CAN, 99.25% are churns and 79.8% of NaNs are churns. This means that each value of 'country code has a differing effect on churn. Hence, this feature has predictive power & we'll keep this column. Let's apply this to the remaining near-constant columns.

In [520]:
# Let's check if the 'country_code' feature have any predictive power
df['label'].replace(churn_numeric, inplace=True)
df['country_code'].fillna('NaN', inplace=True)
df[['country_code', 'label']].groupby(['country_code']).mean()

,label
country_code,
CAN,0.992539
NaN,0.798426


In [521]:
# Similar to the 'country_code' example, let's calculate churn rate for each value of each feature to assess predictive power.
# IGNORE THE ERROR below, since we already have the results for all near constant columns.
df_near_const.fillna('NaN', inplace=True)
df_near_const['label'] = df['label']
for col in df_near_const.columns:
    print(df_near_const[[col, 'label']].groupby([col]).mean())

                 label
country_code          
CAN           0.992539
NaN           0.798426
                  label
currency_code          
CAD            0.790710
USD            0.945965
              label
is_active          
f          0.590285
t          0.814307
                                 label
include_client_consolidation          
f                             0.899642
t                             0.799036
                      label
credit_limit_type          
0.0                0.798691
2.0                1.000000
3.0                1.000000
7.0                1.000000
9.0                1.000000
NaN                1.000000
                          label
dividend_confirm_code          
0.0                    0.796343
1.0                    1.000000
5.0                    1.000000
6.0                    0.000000
7.0                    1.000000
9.0                    1.000000
NaN                    1.000000
                         label
options_trading_type          
0.

ValueError: Grouper for 'label' not 1-dimensional

### 1.3.2.1 Sanity check: Is 'sss_agent' all 100% churn rate?
Woah! It seems that 'sss_agent' is cutoff in the above cell. There are 3816 unique values in 'sss_agent', but 3815 all are 1.0 (100%), the other being NaNs. Hence, 'sss_agent' provides no predictive value. Removed!

In [522]:
# We see that all non-empty values are 1.0, except the NaN
sss = df_near_const[['sss_agent', 'label']].groupby(['sss_agent']).mean()
label_one_count = 0
for row in sss.iterrows():
    if row[1][0] == 1.0:
        label_one_count += 1 
    else:
        print(row)
print(label_one_count)

('NaN', label    0.798144
Name: NaN, dtype: float64)
3815


In [523]:
# Remove the near-constant columns with no predictive power
df.drop(columns=['sss_agent', 'use_mail', 'shareholder_instructions_received', 'is_spousal_transfer',
                 'consent_to_email_delivery'], inplace=True)
df.shape

(673339, 72)

## 1.4 Remove any remaining columns with little to no predictive power
Though most are not applicable (since they're numerical, or too many categories), this is just to make sure we don't include any more unnecessary columns. But after running this, it seems that they all have some predictive power! So we won't remove any columns.

In [524]:
# Let's do the same churn rate analysis for the remaining columns
# First, get all columns we didn't check from df_near_const.columns
remaining_cols = list(set(df.columns).difference(set(df_near_const.columns)))
len(remaining_cols)

# Do the same analysis
df.fillna('NaN', inplace=True)
for col in remaining_cols:
    print(df[[col, 'label']].groupby([col]).mean())

                   label
risk_tolerance          
B10M90          1.000000
B20G70S10       0.000000
B40G60          1.000000
B50G20S30       1.000000
B99             0.666667
...                  ...
M99             0.768833
M99S00          0.000000
N99             0.000000
NaN             0.844417
S99             0.158172

[848 rows x 1 columns]
                          label
is_plan_grandfathered          
NaN                    0.999877
f                      0.621567
                          label
last_maintenance_user          
ABUADJE                0.818713
ACHDJIH                1.000000
AGBANO4                0.500000
AGHASIA                0.655172
AHMADT                 1.000000
...                         ...
ZEKA2                  0.230769
ZEKAK                  0.178571
ZHANGLE                1.000000
ZIMMEAD                1.000000
ZUNIGA2                0.000000

[445 rows x 1 columns]
                label
retail_plan          
LF           0.666845
LI           0.57

# 2. Data Cleaning/Formatting
Now, we just need to format our data to make it ingestible for training and fitting.

The 3 major steps are:
- Impute missing values
- Apply encoding to categorical columns
- Undersampling to balance dataset

## 2.1 Impute missing values for each column
For this, I refered to the value count for each column (shown below).

There are 4 types of imputations involved:
1. **Constant imputation** 
2. **Mode imputation**
3. **Median imputation**
4. **Custom imputation**

In [525]:
# Print all unique values and its count (as reference)
for i in df.columns:
    print(df[i].value_counts())
    print('--------------------------------------------------')
df.replace({'NaN':None}, inplace=True) # revert back

type_code
CASH              190811
REG RRSP          114218
TFSA              108073
CASH SWEEP         80307
RRIF               38902
MRGN               32115
LIRA/LRSP/RLSP     28290
RESP               22726
SPOUSAL RRSP       21885
OFFBOOK            11982
SPOUSAL RRIF        8978
COD                 7748
MISSING             3198
SHORT               1584
RDSP                1137
HYPOTHECATED         445
LIRA                 413
CASHA                249
NaN                  237
HEDGE                 30
LIF/RLIF/LRIF         11
Name: count, dtype: int64
--------------------------------------------------
is_registered
t    344633
f    328706
Name: count, dtype: int64
--------------------------------------------------
country_code
NaN    669050
CAN      4289
Name: count, dtype: int64
--------------------------------------------------
currency_code
CAD    634512
USD     38827
Name: count, dtype: int64
--------------------------------------------------
is_active
t    629323
f     44016
Na

### 2.1.1 Constant imputation
This to provide an extra category called 'Unknown' that represents missingness. This is useful when missing values may carry specific meaning.

In [526]:
# Imputing categorical values with constant 'Unknown'
cols_unknown = ['country_code', 'is_spousal', 'is_arp_locked', 'language_code', 'sss_type', 
                'rep_commission_rate', 'discretionary_trading_authorized', 'iso_funds_code', 
                'shareholder_language', 'dup_trip_quad_code', 'special_tag', 'conjunction', 'function_code', 
                'tms_settlement_location', 'portfolio_cost_method',
                'loan_limit_override', 'interactive_portfolio_code', 
                'non_plan_book_value_flag', 'is_pledged', 'non_calendar_year_end', 'is_resp',
                'use_original_date_for_payment_calc', 'is_family_resp', 'is_hrdc_resp', 
                'retail_last_maintenance_user']
df.loc[:, cols_unknown] = df.loc[:, cols_unknown].fillna('Unknown')

### 2.1.2 Mode imputation
This is simply replacing missing values with the most frequent value of the feature. Applicable for categorical columns.

In [527]:
# Imputing categorical values with mode
cols_mode = ['type_code', 'branch', 'credit_limit_type', 'retail_plan', 'dividend_confirm_code',
             'arp_pension_origin', 'options_trading_type', 'rep_commission_override', 'interest_dividend_conversion_type',
             'guarantee_gtor_type', 'terminal_code', 'mailing_consent', 'risk_tolerance', 'investment_objective',
             'last_maintenance_user', 'number_of_beneficiaries', 'resp_specimen_plan']
for col in cols_mode:
    mode_value = df[col].dropna().mode()[0]
    df.loc[:, col] = df.loc[:, col].fillna(mode_value)

### 2.1.3 Median imputation
These are for numerical columns by calculating the median across the values. Then, there are some custom imputations I made for each business context

In [528]:
# Imputing numerical values with median values
df.replace({'NaN':None}, inplace=True) # revert back
for col in ['deceased_fair_market_value', 'resp_specimen_plan']:
    median_value = df[col].dropna().median()
    df.loc[:, col] = df.loc[:, col].fillna(median_value)

### 2.1.4 Custom imputation
Finally, there are some custom imputations I made for each business context

In [529]:
# Others
df.loc[:, ['class_id', 'contract_type', 'esir_number']] = df.loc[:, ['class_id', 'contract_type', 'esir_number']].fillna(0)
df.loc[:, 'is_midwest_clearing_account'] = df.loc[:, 'is_midwest_clearing_account'].fillna(True)
df.loc[:, 'portfolio_report_option'] = df.loc[:, 'portfolio_report_option'].fillna(1.0)
df.loc[:, 'target_grantor_grantee_flag'] = df.loc[:, 'target_grantor_grantee_flag'].fillna(2.0)
df.loc[:, 'title'] = df.loc[:, 'title'].fillna('None')
df.loc[:, ['has_discrete_auth', 'is_plan_grandfathered']] = df.loc[:, ['has_discrete_auth', 'is_plan_grandfathered']].fillna('t')

### 2.1,5 Side track: Too much categories
There were some non-ordinal categoricals with too much categories (1000+). So for now I dropped them, but will potentially make use, when necessary.

In [530]:
too_many_categories = ['terminal_code', 'title', 'function_code', 'mailing_consent', 'risk_tolerance', 'investment_objective',
                        'last_maintenance_user', 'retail_last_maintenance_user']
garbage_cols = ['tms_settlement_location_DTCPRM', 'tms_settlement_location_MTL', 'special_tag_NOT FOUND', 'loan_limit_override_C', 'conjunction_OR', 'is_pledged_t', 'branch_IND ALLI', 'arp_pension_origin_PE', 'tms_settlement_location_PRM']
df.drop(columns=too_many_categories, inplace=True)

## 2.2 One-hot encoding for non-ordinal categorical features
Basically, given that nearly all the columns are either numerical, non-ordinal categorical or boolean columns. I thought it was most appropriate for one-hot encoding non-ordinal categorical and boolean columns and leave the numerical columns as is, because they are ingestible for training.

I divided the 2 columns into 3 categories, just in case I might come back and do further processing for one of them:
- **bool_cols**: boolean columns; contains only 't' or 'f'
- **bool_cols_nan**: boolean columns with NaN; contains either 't', 'f' or NaN
- **non_ordinal_cols**: the remaining non-ordinal categoricals

For this, I refered to the value count for each column (which I displayed in the cells above).

Just as a reference, these are all the numerical columns, which I left as is.
- class_id, contract_type, credit_limit_type, dividend_confirm_code, options_trading_type, guarantee_gtor_type, deceased_fair_market_value, target_grantor_grantee_flag, esir_number, portfolio_report_option, number_of_beneficiaries, resp_specimen_plan

In [531]:
bool_cols = ['is_registered', 'is_active', 'include_client_consolidation', 'use_client_address', 'use_hand_delivery',
             'share_name_address_to_issuer', 'is_tms_eligible', 'is_agent_bbs_participant', 'has_multiple_name',
             'receive_general_mailings', 'has_discrete_auth', 'is_non_objecting_beneficial_owner',
             'is_objecting_to_disclose_info', 'consent_to_pay_for_mail', 'has_received_instruction', 'is_inventory_account',
             'is_extract_eligible', 'is_plan_grandfathered', 'visible_in_reports']
bool_cols_nan = ['is_spousal', 'is_arp_locked', 'is_midwest_clearing_account', 'discretionary_trading_authorized', 'is_pledged',
                 'is_resp', 'use_original_date_for_payment_calc', 'is_family_resp', 'is_hrdc_resp', 'is_olob']
non_ordinal_cols = ['type_code', 'country_code', 'currency_code', 'branch', 'retail_plan', 'arp_pension_origin',
                    'language_code', 'sss_type', 'rep_commission_rate', 'rep_commission_override',
                    'interest_dividend_conversion_type', 'iso_funds_code', 'shareholder_language', 'dup_trip_quad_code',
                    'special_tag', 'conjunction', 'tms_settlement_location', 'portfolio_cost_method',
                    'loan_limit_override', 'interactive_portfolio_code', 'non_plan_book_value_flag', 'non_calendar_year_end']
combined = bool_cols + bool_cols_nan + non_ordinal_cols
df_encoded = pd.get_dummies(df, columns=combined)

## 2.3 Dealing with unbalanced labels
Given that we have a sufficient amount of data, let's undersample to balance the datasets. This is to prevent bias and overfitting.

In [532]:
# Let's check how balanced the labels are
target_counts = df_encoded['label'].value_counts()
target_counts

label
1    538444
0    134895
Name: count, dtype: int64

In [533]:
# Use RandomUnderSampler to undersample the churn (1) values
df_encoded.drop(columns=['label'], inplace=True)
rus = RandomUnderSampler(random_state=42)
X_balanced, y_balanced = rus.fit_resample(df_encoded, df['label'])
X_balanced.drop(columns=garbage_cols, inplace=True)

# 3. Train and fit data
Given that we have 188 columns of one-hot encoded data, it only makes sense to use some sort of tree-based algorithm like decision tree, because it works well with high-dimensionality. But we can look at other algorithms like ensemble methods (random forest) or gradient-boosted trees (xgboost).

In [534]:
# train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=1)

In [535]:
# Apply decision tree classifier
dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)
y_pred_dtree = dtree.predict(X_test)
report = classification_report(y_test, y_pred_dtree)
print(report)

              precision    recall  f1-score   support

           0       0.96      1.00      0.98     26955
           1       1.00      0.96      0.98     27003

    accuracy                           0.98     53958
   macro avg       0.98      0.98      0.98     53958
weighted avg       0.98      0.98      0.98     53958



In [536]:
# decision tree f1 score
f1 = f1_score(y_test, y_pred_dtree)
print(f"F1 Score for dtree: {f1}")

F1 Score for dtree: 0.9789877993673747


# 4. Dealing with Overfitting
**Overfitting** refers to the condition when the model completely fits the training data but fails to generalize the testing unseen data.

On my first try, I already got a 98% F1-score, which is really high... suspicially high. So I will dedicate this entire step to answer the golden question. **Is our model overfitting or is the model just really accurate?**

There are many ways to deal with it. We've already used methods to prevent overfitting above, even before training the data:
 - Removing nodes with little to no predictive power.
 - Getting rid of irrelevant features
 - Balancing the dataset

But there are other ways for us to further check if the model is overfitting:
- Cross validation
- Ensemble methods
- Adjusting the hyperparameters (especially max_depth)

## 4.1 Cross validation
Since cross validation partitions the data into different training and testing data, we can determine if overfitting occurs by looking at the variance of each result of the partitioned data (in this case, 5 partitions). It seems that each result is nearly identical, all 98%~. Great! Less likely that the model is overfitting.

In [537]:
# decision tree cross validation
cv_scores = cross_val_score(estimator=dtree, X=X_train, y=y_train, cv=5)
print(cv_scores)
print(f"Average CV Score for dtree: {sum(cv_scores)/len(cv_scores)}")

[0.9785021  0.97810828 0.97634713 0.97803827 0.97778344]
Average CV Score for dtree: 0.9777558420498833


## 4.2 Ensemble methods
Why does random forest mitigate overfitting? RF uses a technique called bagging, where multiple decision trees are trained on different samples of the training data. We then average the predictions of multiple trees, RF reduces the variance of the model, hence prevent overfitting.

We see that the F1-score of RF is the exact same as the decision tree classifier, which helps us deduce that it may be less likely that it is indeed overfitting. Great!

In [539]:
# Apply random forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
report = classification_report(y_test, y_pred_rf)
print(report)

              precision    recall  f1-score   support

           0       0.96      1.00      0.98     26955
           1       1.00      0.96      0.98     27003

    accuracy                           0.98     53958
   macro avg       0.98      0.98      0.98     53958
weighted avg       0.98      0.98      0.98     53958



In [540]:
# random forest cross validation
cv_scores = cross_val_score(estimator=rf, X=X_train, y=y_train, cv=5)
print(cv_scores)
print(f"Average CV Score for dtree: {sum(cv_scores)/len(cv_scores)}")

[0.97912757 0.97905808 0.97741278 0.9785016  0.97917342]
Average CV Score for dtree: 0.9786546903319693


## 4.3 Adjusting hyper-parameters
It is definitely a possibility that when we don't put restraints on the decision tree parameters, we allow the tree nodes to grow to its full depth, which may cause overfitting.

### 4.3.1 Analyzing default parameters

First, let's see what the max_depth, min_samples_split, min_samples_leaf look like when we don't put any restraints on the parameters and see how we can adjust it in our hyperparameter tuning.
As we suspected, the depth of the tree is too long and we would like to raise the minimum sample size to split and minimum sample size to be a lead node.

In [541]:
# Check default parameters
print("Max Depth of Decision Tree:", dtree.tree_.max_depth)
print("Min Samples Split:", dtree.min_samples_split)
print("Min Samples Leaf:", dtree.min_samples_leaf)

Max Depth of Decision Tree: 44
Min Samples Split: 2
Min Samples Leaf: 1


### 4.3.2 Sanity check: Will smaller max_depth change F1-score?
Since we know that an overextended decision tree directly causes overfitting, let's see if our default decision tree overextended. A mini-hyperparameter tuning!

It seems that for **max_depth = {10, 20, 30, 40}**, the F1-score stayed the same! Now, we are very confident that this may not overfitting and our model is just performing really well!

In [542]:
# Apply decision tree classifier
for depth in [10, 20, 30, 40]:
    dtree = DecisionTreeClassifier(max_depth=depth)
    dtree.fit(X_train, y_train)
    y_pred = dtree.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    print(f"F1 Score for dtree with max_depth={depth}: {f1}")

F1 Score for dtree with max_depth=10: 0.9805548744836953
F1 Score for dtree with max_depth=20: 0.979575686776953
F1 Score for dtree with max_depth=30: 0.9790431000395413
F1 Score for dtree with max_depth=40: 0.9790599578186201


### 4.3.3 Tuning hyperparameters
Now that we have a general direction on how to adjust these parameters, as in we have some ideas of other numbers for the parameters. Let's try every single combination of these parameters to get the optimal score with adjustment to overfitting.

In [543]:
param_grid = {
    "max_depth": [3,10,20],
    "min_samples_split": [2,5,10],
    "min_samples_leaf": [1,2,5]
}

dtree = DecisionTreeClassifier(random_state=42)
grid_cv = GridSearchCV(dtree, param_grid, scoring="roc_auc", n_jobs=-1, cv=3).fit(X_train, y_train)

print("Param for GS", grid_cv.best_params_)
print("CV score for GS", grid_cv.best_score_)
print("Train AUC ROC Score for GS: ", roc_auc_score(y_train, grid_cv.predict(X_train)))
print("Test AUC ROC Score for GS: ", roc_auc_score(y_test, grid_cv.predict(X_test)))

Param for GS {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10}
CV score for GS 0.9826125933697271
Train AUC ROC Score for GS:  0.9799337403819317
Test AUC ROC Score for GS:  0.9809088691087334


# 5. Export with final model

## 5.1 Train final model with the tuned parameters

In [544]:
dtree = DecisionTreeClassifier(max_depth=10, min_samples_leaf=2, min_samples_split=10)
dtree.fit(X_train, y_train)
y_pred = dtree.predict(X_test)
f1 = f1_score(y_test, y_pred)
print(f"F1 Score for dtree with best params: {f1}")

F1 Score for dtree with best params: 0.9805526737715741


## 5.2 Import and recreate same process for test.csv
Same methodology as above, but all in one cell.

In [545]:
# STEP 1
test_df = pd.read_csv('test.csv')
test_df.drop(columns=['id', 'last_trade_date', 'inception_date', 'last_update_date', 'last_maintenance_time',
                 'plan_effective_date', 'plan_end_date', 'rrif_original_date', 'inserted_at', 'updated_at',
                 'retail_last_maintenance_time'], inplace=True)
test_df.drop(columns=['cashflows_custody_fee'], inplace=True) # completely empty; only constains NaNs
test_df.drop(columns=['net_of_fees', 'custody_fee_withdrawal', 'is_fee_exempt', 'rrsp_limit_reached', 'rrsp_limit_reached',
                 'is_portfolio_account', 'has_no_min_commission', 'is_parameters_account', 'spousal_age_flag',
                 'is_broker_account', 'is_gl_account', 'is_control_account', 'fee_paid_separately', 'special_fee_code',
                'portfolio_name_address_option', 'portfolio_summary_option', 'debit_code', 'sss_location'], inplace=True)
test_df.drop(columns=['sss_agent', 'use_mail', 'shareholder_instructions_received', 'is_spousal_transfer',
                 'consent_to_email_delivery'], inplace=True)

# STEP 2
# Imputing categorical values with constant 'Unknown'
cols_unknown = ['country_code', 'is_spousal', 'is_arp_locked', 'language_code', 'sss_type', 
                'rep_commission_rate', 'discretionary_trading_authorized', 'iso_funds_code', 
                'shareholder_language', 'dup_trip_quad_code', 'special_tag', 'conjunction', 'function_code', 
                'tms_settlement_location', 'portfolio_cost_method',
                'loan_limit_override', 'interactive_portfolio_code', 
                'non_plan_book_value_flag', 'is_pledged', 'non_calendar_year_end', 'is_resp',
                'use_original_date_for_payment_calc', 'is_family_resp', 'is_hrdc_resp', 
                'retail_last_maintenance_user']
test_df.loc[:, cols_unknown] = test_df.loc[:, cols_unknown].fillna('Unknown')
# Imputing categorical values with mode
cols_mode = ['type_code', 'branch', 'credit_limit_type', 'retail_plan', 'dividend_confirm_code',
             'arp_pension_origin', 'options_trading_type', 'rep_commission_override', 'interest_dividend_conversion_type',
             'guarantee_gtor_type', 'terminal_code', 'mailing_consent', 'risk_tolerance', 'investment_objective',
             'last_maintenance_user', 'number_of_beneficiaries', 'resp_specimen_plan']
for col in cols_mode:
    mode_value = test_df[col].dropna().mode()[0]
    test_df.loc[:, col] = test_df.loc[:, col].fillna(mode_value)
# Imputing numerical values with median values
for col in ['deceased_fair_market_value', 'resp_specimen_plan']:
    median_value = test_df[col].dropna().median()
    test_df.loc[:, col] = test_df.loc[:, col].fillna(median_value)
# Others
test_df.loc[:, ['class_id', 'contract_type', 'esir_number']] = test_df.loc[:, ['class_id', 'contract_type', 'esir_number']].fillna(0)
test_df.loc[:, 'is_midwest_clearing_account'] = test_df.loc[:, 'is_midwest_clearing_account'].fillna(True)
test_df.loc[:, 'portfolio_report_option'] = test_df.loc[:, 'portfolio_report_option'].fillna(1.0)
test_df.loc[:, 'target_grantor_grantee_flag'] = test_df.loc[:, 'target_grantor_grantee_flag'].fillna(2.0)
test_df.loc[:, 'title'] = test_df.loc[:, 'title'].fillna('None')
test_df.loc[:, ['has_discrete_auth', 'is_plan_grandfathered']] = test_df.loc[:, ['has_discrete_auth', 'is_plan_grandfathered']].fillna('t')

too_many_categories = ['terminal_code', 'title', 'function_code', 'mailing_consent', 'risk_tolerance', 'investment_objective',
                        'last_maintenance_user', 'retail_last_maintenance_user']
test_df.drop(columns=too_many_categories, inplace=True)

# one-hot encoding
bool_cols = ['is_registered', 'is_active', 'include_client_consolidation', 'use_client_address', 'use_hand_delivery',
             'share_name_address_to_issuer', 'is_tms_eligible', 'is_agent_bbs_participant', 'has_multiple_name',
             'receive_general_mailings', 'has_discrete_auth', 'is_non_objecting_beneficial_owner',
             'is_objecting_to_disclose_info', 'consent_to_pay_for_mail', 'has_received_instruction', 'is_inventory_account',
             'is_extract_eligible', 'is_plan_grandfathered', 'visible_in_reports']
bool_cols_nan = ['is_spousal', 'is_arp_locked', 'is_midwest_clearing_account', 'discretionary_trading_authorized', 'is_pledged',
                 'is_resp', 'use_original_date_for_payment_calc', 'is_family_resp', 'is_hrdc_resp', 'is_olob']
non_ordinal_cols = ['type_code', 'country_code', 'currency_code', 'branch', 'retail_plan', 'arp_pension_origin',
                    'language_code', 'sss_type', 'rep_commission_rate', 'rep_commission_override',
                    'interest_dividend_conversion_type', 'iso_funds_code', 'shareholder_language', 'dup_trip_quad_code',
                    'special_tag', 'conjunction', 'tms_settlement_location', 'portfolio_cost_method',
                    'loan_limit_override',
                    'interactive_portfolio_code', 'non_plan_book_value_flag', 'non_calendar_year_end']
combined = bool_cols + bool_cols_nan + non_ordinal_cols
test_df_encoded = pd.get_dummies(test_df, columns=combined)

## 5.3 Predict labels for test.csv

In [546]:
df_id = pd.read_csv('test.csv')
predictions = dtree.predict(test_df_encoded)
predictions_mapped = ['Churn' if pred == 1 else 'No Churn' for pred in predictions]
result_df = pd.DataFrame({'id': df_id['id'], 'predicted': predictions_mapped})
result_df.to_csv('predictions.csv', index=False)